In [1]:
import pandas as pd
import numpy as np 
import os.path as op

from necessity_functions import NecessityUnivariate, NecessityBivariate
from robust_set_sampling import RobustCredalSetUnivariate, RobustCredalSetBivariate
from ordering_focal_set_importance import random_generator_poss
from copulas import min_copula

### Declare marginal possibilities and orders

In [2]:
a = {'x1':0.2, "x2":1, "x3":0.2}
order_a = pd.DataFrame(data=[2,1], index=["x2", "x1,x2,x3"], columns=["order"])
order_a_p = pd.DataFrame(data=[1,2,3], index=["x1","x2","x3"], columns=["order"])

b = {'y1':0.7, "y2":1}
order_b = pd.DataFrame(data=[1,2], index=["y2", "y1,y2"], columns=["order"])
order_b_p = pd.DataFrame(data=[1,2], index=["y1","y2"], columns=["order"])

### Necessity functions

In [3]:
nec_x = NecessityUnivariate(a, order_a)
nec_y = NecessityUnivariate(b, order_b)

nec_xy = NecessityBivariate(nec_x, nec_y, min_copula)

In [4]:
rob_x = RobustCredalSetUnivariate(nec_x)
rob_y = RobustCredalSetUnivariate(nec_y)

rob_xy = RobustCredalSetBivariate(rob_x, rob_y, order_a_p, order_b_p, min_copula)

# Other tests

In [2]:
gen = random_generator_poss(["x1", "x2", "x3", "x4"])

In [3]:
i=0
for k in gen:
    print(k)
    i+=1
    if i == 30:
        break

{'x1': 0.14, 'x2': 0.95, 'x3': 0.31, 'x4': 1.0}
{'x1': 0.83, 'x2': 0.41, 'x3': 0.55, 'x4': 1.0}
{'x1': 0.54, 'x2': 0.33, 'x3': 0.79, 'x4': 1.0}
{'x1': 0.45, 'x2': 0.13, 'x3': 0.4, 'x4': 1.0}
{'x1': 0.75, 'x2': 0.28, 'x3': 1.0, 'x4': 0.49}
{'x1': 1.0, 'x2': 0.54, 'x3': 1.0, 'x4': 1.0}
{'x1': 0.97, 'x2': 0.52, 'x3': 1.0, 'x4': 0.12}
{'x1': 1.0, 'x2': 0.78, 'x3': 0.61, 'x4': 0.92}
{'x1': 0.46, 'x2': 0.06, 'x3': 1.0, 'x4': 0.64}
{'x1': 1.0, 'x2': 1.0, 'x3': 1.0, 'x4': 0.84}
{'x1': 0.75, 'x2': 0.15, 'x3': 1.0, 'x4': 0.82}
{'x1': 0.79, 'x2': 0.19, 'x3': 1.0, 'x4': 0.8}
{'x1': 0.86, 'x2': 1.0, 'x3': 0.86, 'x4': 0.88}
{'x1': 1.0, 'x2': 0.27, 'x3': 0.01, 'x4': 0.65}
{'x1': 0.22, 'x2': 1.0, 'x3': 0.64, 'x4': 1.0}
{'x1': 1.0, 'x2': 0.15, 'x3': 0.48, 'x4': 1.0}
{'x1': 0.67, 'x2': 1.0, 'x3': 1.0, 'x4': 1.0}
{'x1': 1.0, 'x2': 0.66, 'x3': 1.0, 'x4': 1.0}
{'x1': 0.21, 'x2': 0.83, 'x3': 1.0, 'x4': 0.06}
{'x1': 0.38, 'x2': 1.0, 'x3': 1.0, 'x4': 0.32}
{'x1': 0.4, 'x2': 0.01, 'x3': 1.0, 'x4': 0.26}
{'x1':